In [51]:
#IMPORTS
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import scipy as sp
import warnings
warnings.filterwarnings('ignore')
pd.set_option("display.max_columns", None)
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
## ASSIGNING DATASET TO A VARIABLE
df = pd.read_csv('../input/carsforsale/cars_raw.csv')
data = df

## METHOD TO STANDARDIZE CAR COLOR NAMES INTO MORE SIMPLER CATEGORIES
def color(x):
    ## IF THE COLOR IS CLOSE TO RED THE COLOR OF THE CAR IS TURNED TO RED
    if 'red' in x.lower() or 'ruby' in x.lower() or 'lava' in x.lower() or 'rouge' in x.lower() or 'raspberry' in x.lower():
        return 'Red'
    ## IF THE COLOR IS CLOSE TO BLACK THE COLOR OF THE CAR IS TURNED TO BLACK
    elif 'black' in x.lower() or 'shadow' in x.lower() or 'ebony' in x.lower() or 'lead' in x.lower():
        return 'Black'
    ## IF THE COLOR IS CLOSE TO WHITE THE COLOR OF THE CAR IS TURNED TO WHITE
    elif 'white' in x.lower() or 'blizzard' in x.lower() or 'ice' in x.lower() or 'moon' in x.lower() or 'lunar' in x.lower():
        return 'White'
    ## IF THE COLOR IS CLOSE TO BLUE THE COLOR OF THE CAR IS TURNED TO BLUE
    elif 'blue' in x.lower() or 'aqua' in x.lower():
        return 'Blue'
    ## IF THE COLOR IS CLOSE TO GREEN THE COLOR OF THE CAR IS TURNED TO GREEN
    elif 'green' in x.lower() or 'moss' in x.lower() or 'olive' in x.lower():
        return 'Green'
    ## IF THE COLOR IS CLOSE TO BROWN THE COLOR OF THE CAR IS TURNED TO BROWN
    elif 'brown' in x.lower() or 'bronze' in x.lower() or 'walnut' in x.lower() or 'sand' in x.lower() or 'mocha' in x.lower():
        return 'Brown'
    ## IF THE COLOR IS CLOSE TO ORANGE THE COLOR OF THE CAR IS TURNED TO ORANGE
    elif 'orange' in x.lower():
        return 'Orange'
    ## IF THE COLOR IS CLOSE TO YELLOW THE COLOR OF THE CAR IS TURNED TO YELLOW
    elif 'yellow' in x.lower():
        return 'Yellow'
    ## IF THE COLOR IS CLOSE TO GRAY THE COLOR OF THE CAR IS TURNED TO GRAY
    elif 'gray' in x.lower() or 'silver' in x.lower() or 'grey' in x.lower() or 'steel' in x.lower() or 'granite' in x.lower() or 'gun' in x.lower() or 'magnetic' in x.lower() or 'cement' in x.lower():
        return 'Gray'
    ## IF THE COLOR CAN'T BE IDENTIFIED, IT IS PLACED IN THE OTHER CATEGORY
    else:
        return 'Other'

##CODE TO CONVERT THE PRICE STRINGS INTO INTEGERS
##GETS RID OF DOLLAR SIGN AND COMMAS 
data['Price'] = data['Price'].str.replace('$','')
data['Price'] = data['Price'].str.replace(',','')
## ELIMINATES ANYTHING WITH NO PRICE
data = data[data['Price'] != 'Not Priced']
## TURNS DATA INTO INTEGER
data['Price'] = data['Price'].astype(int)
##RUNS COLOR ASSIGNMENT FUNCTION
data['ExteriorColor'] = data['ExteriorColor'].apply(color)

##CODE T0 FILTER DATA SO ONLY TOP 25% OF DATA IS USED TO CALCULATE PREFERENCES
##CALCULATES MEAN REVIEW RATING
mean = data['ConsumerRating'].mean()
##CALCUATES MINIMUM NUMBER OF REVIEWS NEEDED TO BE CONSIDERD
minimum = round(data['ConsumerReviews'].mean())
##CALCULATION FOR EACH CARS WEIGHTED RATING
data['WeightedConsumerReview'] = data.ConsumerReviews/(data.ConsumerReviews + minimum) * data.ConsumerRating +(minimum/(data.ConsumerReviews + minimum))*mean
##CALCULATION FOR THE AVERAGES  OF ALL CATEGORICAL RANKINGS IN A CAR AND IS ADDED TO THE NEW ADDED RATINGS
data['AverageRatings'] = (data.ComfortRating + data.InteriorDesignRating + data.PerformanceRating + data.ValueForMoneyRating + data.ExteriorStylingRating + data.ReliabilityRating)/6
data = data.sort_values(['WeightedConsumerReview','AverageRatings'],ascending=False).drop(columns=['WeightedConsumerReview','AverageRatings'])
##FILTERS LIST TO TOP 75TH PERCENTILE
y = int(round(len(data.index)*.25))
dataFiltered = data.head(y)

## PART 1 - PREFERENCES IN CAR BETWEEN RESIDENTS IN EACH STATE
## LIST FOR ALL THE ATTRIBUTES I WANT PRINTED OUT
attributes = ["Year", "Make", "Model", "Drivetrain", "MaxMPG", "MinMPG", "FuelType", "Transmission", "Engine", "Mileage" ]
##PRINTS OUT ATTRIBUTES FOR EACH STATE
states = pd.unique(dataFiltered["State"].values.ravel())
for state in states:
    print("Most popular/favorable attributes of cars of citizens in ", state)
    ##PRINTS ALL WANTED ATTRIBUTES FOR A STATE
    for attr in attributes:
        ## PRINTS ONLY THE MOST COMMON/PREFFERRED CAR TRAIT FOR THE STATE
        for value, count in dataFiltered.loc[dataFiltered["State"] == state][attr].value_counts().iteritems():
            if count > 1:
                print("     ",attr, "   -   ", value)
                break

## PART 2 - FINDING THE MOST EXPENSIVE CAR BASED ON COLOR
## DIVIDES PLOTS INTO TWO SEPARATE PLOTS
fig, axs = plt.subplots(ncols=2)
##MAKES A BOXPLOT TO FIND THE AVERAGE MEDIAN CAR PRICE BY COLOR
sns.boxplot(x= data["ExteriorColor"], y = data["Price"], ax = axs[0]).set(title='Average Median Price of Car for Each Color')
##MAKES A BOXPLOT TO FIND THE AVERAGE MEAN CAR PRICE BY COLOR
data = data.groupby(data["ExteriorColor"], as_index=False)['Price'].mean()
sns.barplot(data["ExteriorColor"], data["Price"], data = data, ci = False, ax = axs[1]).set(title='Average Mean Price of Car for Each Color')